## Stochastic gradient descent-based inference for dynamic network models with attractors
## This script analyzes Twitter congressional hashtag networks with 207 nodes. These nodes represent members consistently present throughout the study period.

In [ ]:
from utils import preprocess_2, ClsnaModel_2,visualize_membership
import numpy as np
import torch
from scipy.linalg import orthogonal_procrustes

In [ ]:
# Set device for computation (GPU if available)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = "cpu"

In [ ]:
# Set global variables for the model
N = 207
DIM = 2
T = 11
SIGMA = 10
TAU = 1

In [ ]:
# Preprocess data
from scipy.spatial.distance import squareform
from sklearn.preprocessing import normalize
from scipy.sparse.csgraph import laplacian
Y = np.load("Y.npy")     
membership = np.load("pi.npy")   
membership = 1*(membership!='D')
s = membership
y_t = squareform(Y[:,:,0])
y = [y_t]
Aw = []
Aw2 = []
Ab = []
for i in range(1,T):
    #normalized graph laplalcian by row
    Aw_t = squareform(y_t)*(np.outer(s, s))
    Aw_t = -2*normalize(laplacian(Aw_t), axis=1, norm='l1')
    Aw2_t = squareform(y_t)*(np.outer(1-s, 1-s))
    Aw2_t = -2*normalize(laplacian(Aw2_t), axis=1, norm='l1')
    Ab_t = squareform(y_t)*(np.outer(1-s, s)+np.outer(s, 1-s))
    Ab_t = -2*normalize(laplacian(Ab_t), axis=1, norm='l1')
    y_t = squareform(Y[:,:,i])
    #save
    y.append(y_t)
    Aw.append(Aw_t)
    Aw2.append(Aw2_t)
    Ab.append(Ab_t)

In [ ]:
density = []
for network in y:
    density.append(np.mean(network))

In [ ]:
density

In [ ]:
# np.sum(membership),len(membership)-np.sum(membership)

In [ ]:
# z = np.concatenate(z)
label, persist, Aw, Aw2, Ab, combination_N=preprocess_2(y, Aw, Aw2, Ab, N, T)

In [ ]:
_s = torch.arange(0,N*(T-1), requires_grad = False)
ar_pair = torch.stack((_s,_s+N), dim = 1)

In [ ]:
combination_N = combination_N.to(device)
label = label.to(device)
persist = persist.to(device)

In [ ]:
LR = 5e-3
MOM = 0.99
LR_P = 1e-2

In [ ]:
#train the model
def train(optimizer, index=None, fixed=None):
    t_index=torch.arange(start=0,end=N*T,device=device,requires_grad=False)
    optimizer.zero_grad()
    loss = model.loss(device=device,label=label,persist=persist,sample_edge=combination_N,T_index=t_index,ss=SIGMA,tt=TAU)
    loss.backward()
    model.para.grad = 0.1*((model.para.grad>0).bool().float()-0.5)
    optimizer.step()
    if index is not None:
        with torch.no_grad():
            model.para[index[0],index[1]] = fixed
    return loss.item()

In [ ]:
#run the optimization process
def run(optimizer,index=None,fixed=None):
    for epoch in range(1,12000):
        loss = train(optimizer=optimizer,index=index,fixed=fixed)
        if epoch%1000 == 0:
            print(loss)
    return loss

# Step 1

In [ ]:
print("Step 1: Fitting initial CLSNA model with higher-dimensional space...")

In [ ]:
# Initialize and train the first model
model = ClsnaModel_2(device,N,T,ar_pair,Aw,Aw2,Ab,D=3).to(device)

In [ ]:
optimizer = torch.optim.SGD([
    {'params': model.z, "momentum": MOM, "lr": LR},
    {'params': model.para, "momentum": 0.0, "lr":LR_P}
    ])

In [ ]:
run(optimizer)

In [ ]:
# Perform PCA to reduce dimensionality
PCA_p = torch.pca_lowrank(model.z.cpu())[2][:,[0,1]]
init_z = (model.z.cpu().detach()@PCA_p).detach().numpy()
init_para = model.para.detach().cpu().numpy()

# Step 2

In [ ]:
print("Step 2: Fitting CLSNA model with targeted dimension and estimating model parameters...")

In [ ]:
# init_z = np.load('../compare/z2_compare.npy')

In [ ]:
model = ClsnaModel_2(device,N,T,ar_pair,Aw,Aw2,Ab,D=2).to(device)
with torch.no_grad():       
    model.z[:,:] = torch.from_numpy(init_z).to(device)
    model.para[:,:] = torch.from_numpy(init_para).to(device)

In [ ]:
optimizer = torch.optim.SGD([
    {'params': model.z, "momentum": MOM, "lr": LR},
    {'params': model.para, "momentum": 0.0, "lr":LR_P}
    ])

In [ ]:
run(optimizer)

In [ ]:
init_z = model.z.cpu().detach().numpy()
init_para = model.para.detach().cpu().numpy()

In [ ]:
# np.save('../compare/z1.npy', init_z)

In [ ]:
membership = np.load("pi.npy")
membership = membership!='D'

In [ ]:
for ti in range(T):
    visualize_membership(z=init_z,membership=np.tile(membership,T),start=ti*N,end=(ti+1)*N)

# Step 3

In [ ]:
print("Step 3: Performing variance/covariance estimation for the parameters of interest...")

In [ ]:
model = ClsnaModel_2(device,N,T,ar_pair,Aw,Aw2,Ab,D=2).to(device)
with torch.no_grad():       
    model.z[:,:] = torch.from_numpy(init_z).to(device)
    model.para[:,:] = torch.from_numpy(init_para).to(device)
optimizer = torch.optim.SGD([
    {'params': model.z, "momentum": MOM, "lr": LR},
    {'params': model.para, "momentum": 0.0, "lr":LR_P}
    ])    
logL = train(optimizer)

In [ ]:
delta_var = 0.1

In [ ]:
def run(optimizer,index=None,fixed=None):
    for epoch in range(1,7000):
        loss = train(optimizer=optimizer,index=index,fixed=fixed)
        if epoch%1000 == 0:
            print(loss)
    return loss

In [ ]:
# Estimate variance/covariance for each parameter
parad = {'alpha':(0,1),'delta':(2,1),'gw':(1,1),'gw2':(3,0),'gb':(2,0)}
var_list = []
cov_list = []

for key, value in parad.items():
    model = ClsnaModel_2(device,N,T,ar_pair,Aw,Aw2,Ab,D=2).to(device)
    with torch.no_grad():       
        model.z[:,:] = torch.from_numpy(init_z).to(device)
        model.para[:,:] = torch.from_numpy(init_para).to(device)
    optimizer = torch.optim.SGD([
    {'params': model.z, "momentum": MOM, "lr": LR},
    {'params': model.para, "momentum": 0.0, "lr":LR_P}
    ])
    newlogL=run(optimizer,value,init_para[value[0],value[1]]+delta_var)
    var_hat = delta_var/(newlogL-logL)**0.5/2**0.5
    var_list.append(round(var_hat,5))
    
    diff = model.para-torch.from_numpy(init_para).to(device)
    extracted_values = [diff[value[0], value[1]].item() for value in parad.values()]
    extracted_values = np.array(extracted_values)
    cov_list.append(extracted_values/delta_var*var_hat**2)

In [ ]:
init_para = init_para.round(3)
printdict = {'a':init_para[0,1],'d':init_para[2,1],'gw':init_para[1,1],'gw2':init_para[3,0],'gb':init_para[2,0]}

In [ ]:
var_list

In [ ]:
printdict

In [ ]:
import csv
fields=var_list
with open('var001', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(fields)

In [ ]:
import csv
fields=list(printdict.values())
with open('theta001', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(fields)